In [1]:
#Sklearn
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

#PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset


import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
class PVDATATIME(Dataset):
  def __init__(self, csv_path, norm = True):
    if norm == True:
      from sklearn.preprocessing import StandardScaler
      self.data = pd.read_csv(csv_path)
      scaler = StandardScaler()
      self.data = scaler.fit_transform(self.data)
    else:
      self.data = pd.read_csv(csv_path)

  def __getitem__(self,idx):
    features = self.data[idx][0:10]
    label = self.data[idx][-1:]

    #Tensor

    features = torch.from_numpy(features.astype(np.float32))
    label = torch.from_numpy(label.astype(np.float32))

    return features, label

  def __len__(self):
    return len(self.data)

## Rocha

In [0]:
class RochaANN(nn.Module):
  def __init__(self, in_f,h1 = 30):
    super(RochaANN, self).__init__()

    #In_f -> h1 (Firt Connect)
    self.fc1 = nn.Linear(in_f,h1)

    # h1 -> out (2°)
    self.out = nn.Linear(h1, 1)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.out(x)
    return x

In [0]:
train_path = '/content/drive/My Drive/IC Victor/Projeto EUA/1 minuto/Banco de dados/pv_1min_train_ghi'
test_path = '/content/drive/My Drive/IC Victor/Projeto EUA/1 minuto/Banco de dados/pv_1min_test_ghi'

train_dataset = PVDATATIME(train_path)
test_dataset = PVDATATIME(test_path)

print(f'Q. de dados para treino: {train_dataset.__len__()}')
print(f'Q. de dados para teste: {test_dataset.__len__()}')

In [0]:
train_loader = DataLoader(train_dataset,
                          batch_size = 100,
                          shuffle = False,
                          num_workers = 4) 

test_loader = DataLoader(test_dataset,
                         batch_size = 100,
                         shuffle = False,
                         num_workers = 4)

In [0]:
#Inicializando criterio, otimizador e rede.
device = torch.device('cuda')
epochs = 500

in_f = len(train_dataset[0][0])
h1 = 30

model = RochaANN(in_f,h1).to(device)

criterion = nn.MSELoss().to(device)


optimizer= torch.optim.SGD(model.parameters(),
                             lr = 1e-4,
                             momentum = 0.9,
                             weight_decay = 5e-4)

NameError: ignored

In [0]:
#Treino

model.train()
training_loss = []
optimizer.zero_grad()
start_time = time.time()
for epoch in range(epochs):
  print(f'Epoca : {epoch}')

  for b, (X_train, Y_train) in enumerate(train_loader):
    #Cast dos dados.
    X_train = X_train.to(device)
    Y_train = Y_train.to(device)

    #Treino
    y_pred = model(X_train)
    loss = torch.sqrt(criterion(y_pred,Y_train))

    training_loss.append(loss.cpu().data)


    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


  
    
    if b % 250 == 0:
      print(f'\t Training batch {b} \t loss: {loss.cpu().data}')

    if epoch % 1000 == 0:
      torch.save(model.state_dict(),'/content/drive/My Drive/IC Victor/Projeto EUA/1 minuto/Rocha1minGHI.pt')



final_time = time.time() - start_time

training_loss = np.asarray(training_loss)

print(f'Tempo:{final_time / 60} minutos')

Streaming output truncated to the last 5000 lines.
	 Training batch 8000 	 loss: 0.010995270684361458
	 Training batch 8250 	 loss: 0.22504526376724243
	 Training batch 8500 	 loss: 0.27745285630226135
	 Training batch 8750 	 loss: 0.0071139102801680565
	 Training batch 9000 	 loss: 0.48265135288238525
	 Training batch 9250 	 loss: 0.016937559470534325
	 Training batch 9500 	 loss: 0.0026530749164521694
	 Training batch 9750 	 loss: 0.06977564096450806
	 Training batch 10000 	 loss: 0.03539352864027023
	 Training batch 10250 	 loss: 0.005390300415456295
Epoca : 384
	 Training batch 0 	 loss: 0.02802584134042263
	 Training batch 250 	 loss: 0.01592271775007248
	 Training batch 500 	 loss: 0.001092903665266931
	 Training batch 750 	 loss: 0.54256671667099
	 Training batch 1000 	 loss: 0.020954666659235954
	 Training batch 1250 	 loss: 0.19406287372112274
	 Training batch 1500 	 loss: 0.14543046057224274
	 Training batch 1750 	 loss: 0.020014846697449684
	 Training batch 2000 	 loss: 0.14

In [0]:
#Teste
#Load Model
model.load_state_dict(torch.load('/content/drive/My Drive/IC Victor/Projeto EUA/1 minuto/Rocha1minGHI.pt'))

print('Modelo carregado com sucesso!')

test_loss = []
score = []

with torch.no_grad():
  model.eval()
  print('Inicio do teste')
  for b, (X_test, y_test) in enumerate(test_loader):

    X_test = X_test.to(device)
    y_test = y_test.to(device)

    y_val = model(X_test)
    loss = torch.sqrt(criterion(y_val,y_test))

    test_loss.append(loss.cpu().data)

    score.append(r2_score(y_test.cpu().numpy(),y_val.cpu().numpy()))

  test_loss = np.asarray(test_loss)
  score = np.asarray(score)

  print(f'RMSE: {test_loss.mean()}, \n Score: {score.mean()}')


## Adel

In [0]:
class AdelANN(nn.Module):
  def __init__(self, in_f,h1 = 11, h2 = 17, h3 = 24):
    super(AdelANN, self).__init__()

    #In_f -> h1 (Firt Connect)
    self.fc1 = nn.Linear(in_f,h1)

    # h1 -> h2 (2°)
    self.fc2 = nn.Linear(h1, h2)

    #h2 -> h3(3°)
    self.fc3 = nn.Linear(h2,h3)

    #h3 -> out (4°)
    self.out = nn.Linear(h3,1)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))


    x = self.out(x)
    return x

In [0]:
#Inicializando criterio, otimizador e rede.
device = torch.device('cuda')
epochs = 10000

in_f = len(train_dataset[0][0])
h1 = 11
h2 = 17
h3 = 24

model = AdelANN(in_f,h1,h2,h3).to(device)

criterion = nn.MSELoss().to(device)


optimizer= torch.optim.SGD(model.parameters(),
                             lr = 1e-4,
                             momentum = 0.9,
                             weight_decay = 5e-4)

In [0]:
model.train()
training_loss = []
optimizer.zero_grad()
start_time = time.time()
for epoch in range(epochs):
  print(f'Epoca : {epoch}')

  for b, (X_train, Y_train) in enumerate(train_loader):
    #Cast dos dados.
    X_train = X_train.to(device)
    Y_train = Y_train.to(device)

    #Treino
    y_pred = model(X_train)
    loss = torch.sqrt(criterion(y_pred,Y_train))

    training_loss.append(loss.cpu().data)


    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


  
    
    if b % 250 == 0:
      print(f'\t Training batch {b} \t loss: {loss.cpu().data}')

    if epoch % 1000 == 0:
      torch.save(model.state_dict(),'/content/drive/My Drive/IC Victor/Projeto EUA/1 minuto/Adel1minGHI.pt')



final_time = time.time() - start_time

training_loss = np.asarray(training_loss)

print(f'Tempo:{final_time / 60} minutos')

Streaming output truncated to the last 5000 lines.
	 Training batch 4750 	 loss: 0.03198619559407234
	 Training batch 5000 	 loss: 0.09304354339838028
	 Training batch 5250 	 loss: 0.00035248391213826835
	 Training batch 5500 	 loss: 0.04304429888725281
	 Training batch 5750 	 loss: 0.0035660366993397474
	 Training batch 6000 	 loss: 0.0032361040357500315
	 Training batch 6250 	 loss: 0.4724712669849396
	 Training batch 6500 	 loss: 1.836593946791254e-05
	 Training batch 6750 	 loss: 0.04711133614182472
	 Training batch 7000 	 loss: 0.7854309678077698
	 Training batch 7250 	 loss: 0.0014602035516873002
	 Training batch 7500 	 loss: 0.07682923227548599
	 Training batch 7750 	 loss: 0.08383543789386749
	 Training batch 8000 	 loss: 0.007627227809280157
	 Training batch 8250 	 loss: 0.2058391273021698
	 Training batch 8500 	 loss: 0.25884419679641724
	 Training batch 8750 	 loss: 0.0011682831682264805
	 Training batch 9000 	 loss: 0.3477537930011749
	 Training batch 9250 	 loss: 0.0023375

KeyboardInterrupt: ignored

In [0]:
#Load Model
model.load_state_dict(torch.load('/content/drive/My Drive/IC Victor/Projeto EUA/1 minuto/Adel1minGHI.pt''))

print('Modelo carregado com sucesso!')

test_loss = []
score = []

with torch.no_grad():
  model.eval()
  print('Inicio do teste')
  for b, (X_test, y_test) in enumerate(test_loader):

    X_test = X_test.to(device)
    y_test = y_test.to(device)

    y_val = model(X_test)
    loss = torch.sqrt(criterion(y_val,y_test))

    test_loss.append(loss.cpu().data)

    score.append(r2_score(y_test.cpu().numpy(),y_val.cpu().numpy()))

  test_loss = np.asarray(test_loss)
  score = np.asarray(score)

  print(f'RMSE: {test_loss.mean()}, \n Score: {score.mean()}')

